In [2]:
import numpy as np
from numpy.linalg import det, eig
import torch
import pdb

In [25]:
alpha=.5
x0=np.random.randn(2)
A=np.identity(2)
b=np.ones(2)
pk=-(b+.5*(A+A.T)@x0)
print('gradients')
print(b+.5*(A+A.T)@x0)
print(x0+1)

gradients
[-0.07622352  1.12531427]
[-0.07622352  1.12531427]


In [22]:
print((x0+1)@(x0+1))
print(x0@x0+2*(x0@b)+2)

2.3986594070218814
2.398659407021881


In [97]:
torch.manual_seed(0)
alpha=1.0
x0=np.random.randn(2)
A=np.identity(2)
b=np.ones(2)
pk=-(b+.5*(A+A.T)@x0)


x1=x0+alpha*pk

n=2
A1=np.random.randn(n,n)
#check equivalence
print(x1@x1)
print(x0@x0+2*(x0@(alpha*pk))+(alpha**2*(pk@pk)))
print()

#base calculation
print(x1@A1@x1)

#expanded calculation 1
ans1=0
for i in range(n):
    for j in range(n):
        ans1+=A1[i, j]*((x0+alpha*pk)[i]*(x0+alpha*pk)[j])
        
print(ans1)

#expanded calculation 2
ans2=0
for i in range(n):
    for j in range(n):
        ans2+=A1[i,j]*((x0[i]*x0[j])+(x0[i]*alpha*pk[j])+(x0[j]*alpha*pk[i])+(alpha**2*pk[i]*pk[j]))

print(ans2)
print()


print('gradient guess 1')
#gradient guess 1
ans2=0
for i in range(n):
    for j in range(n):
        ans2+=A1[i,j]*((x0[i]*pk[j])+(x0[j]*pk[i])+(2*alpha*pk[i]*pk[j]))
print(ans2)

print('gradient guess 2')
#gradient guess 2
ans2=(A1.T + A1)@(x1)@pk
print(ans2)


#calculate true gradient
x1_torch=torch.tensor(x0+alpha*pk)
x0_torch=torch.tensor(x0)
alpha=torch.tensor(alpha, requires_grad=True)
n=2
A1_torch=torch.tensor(A1, requires_grad=True)
ans1=torch.tensor(0.0)
pk=torch.tensor(pk)
'''
for i in range(n):
    for j in range(n):
        ans1+=A1_torch[i, j]*((x0_torch+alpha*pk)[i]*(x0_torch+alpha*pk)[j])
'''

#print(x1_torch@A1_torch@x1_torch)
ans1=(x0_torch+alpha*pk)@A1_torch@(x0_torch+alpha*pk)
ans1.backward()
print()
print(f'ans: {ans1}')
print(f'grad: {alpha.grad}')

2.0
2.0

-0.9393977723368696
-0.9393977723368696
-0.9393977723368695

gradient guess 1
-2.9205672699507716
gradient guess 2
-2.9205672699507725

ans: -0.9393977723368696
grad: -2.920567512512207


# Full Equation

In [4]:
#guess
alpha=1.0
x0=np.random.randn(2)
A=np.identity(2)
b=np.ones(2)
pk=-(b+.5*(A+A.T)@x0)
x1=x0+alpha*pk
n=2
A1=np.random.randn(n,n)


print('guess')
print(b@pk+.5*((A1.T+A1)@(x1)).T@pk)


#calculate true gradient
x1_torch=torch.tensor(x0+alpha*pk, requires_grad=True)
x0_torch=torch.tensor(x0, requires_grad=True)
alpha=torch.tensor(alpha, requires_grad=True)
n=2
A1_torch=torch.tensor(A1, requires_grad=True)
pk=torch.tensor(pk, requires_grad=True)
b_torch=torch.tensor(b, requires_grad=True)

ans1=b_torch@(x0_torch+alpha*pk)+.5*(x0_torch+alpha*pk)@A1_torch@(x0_torch+alpha*pk)

print(f'ans: {ans1}')
ans1.backward()
print(f'grad: {alpha.grad}')

guess
1.5596563047955918
ans: -0.7013268760344233
grad: 1.5596561431884766


# plug $\alpha^*$ into gradient

In [27]:
#guess
alpha=1.0
b=np.random.randn(n)
A=np.identity(2)
#A=b[:, None]@b[None]
n=2
x0=np.array([4,2])
pk=-(b+.5*(A+A.T)@x0)
x1=x0+alpha*pk
alpha_star=(pk@pk/(pk@A@pk))


alpha=alpha_star
print(b@pk+(A@(x0+alpha*pk))@pk)


#make symmetric positive definite matrix
b=np.random.randn(n)


#Try for some positive semidefinite matrix
A1=np.random.randn(n,n)
A1=A1@A1.T
val, vec = eig(A1)
while any(val<=0):
    A1=np.random.randn(n,n)
    A1=A1@A1.T
    val, vec = eig(A1)

pk=-(b+.5*(A1+A1.T)@x0)

print(f'eigenvalues: {val}')
alpha_star=((pk@pk)/(pk@A1@pk))
alpha=alpha_star
print(b@pk+(A1@(x0+alpha*pk))@pk)


print(b@pk+pk@A1@x0+pk@pk)
print((A@x0-b)@-(A@x0-b))

1.5543122344752192e-15
eigenvalues: [3.1012312  0.92890586]
6.661338147750939e-15
2.842170943040401e-14
-21.29008087464879
